In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

# from datasets import load_dataset

# dataset = load_dataset("yelp_review_full")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
import pickle
# with open('yelp_dataset.pkl', 'wb') as file:
#     pickle.dump(dataset, file)

In [3]:
with open('yelp_dataset.pkl', 'rb') as file:
    dataset = pickle.load(file)

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})


In [5]:
train_dataset = dataset['train']
test_dataset = dataset['test']

# Convert train and test datasets to arrays
train_data = train_dataset['text']
train_labels = train_dataset['label']
test_data = test_dataset['text']
test_labels = test_dataset['label']

# Convert labels to lists (optional)
# train_labels = train_labels.tolist()
# test_labels = test_labels.tolist()

In [6]:
print(train_data[0])
print(train_labels[0])


dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.
4


## Preprocess

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/revanthgundam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/revanthgundam/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
import sys  
!{sys.executable} -m pip install contractions
import contractions

In [9]:
# import re  # Import the regular expressions module

# # use tokenizer to remove punctuation

# def tokenize_text(text):
#     # Replace "!" with "exm" using regular expressions
#     text = re.sub(r'!', ' exm', text)
    
#     expanded_words = []
#     for word in text.split():
#         # using contractions.fix to expand the shortened words
#         expanded_words.append(contractions.fix(word))   
    
#     expanded_text = ' '.join(expanded_words)
#     # print(expanded_text)
    
#     text = expanded_text    
    
#     # replace a-b with a and b
#     text = text.replace('-', ' ')
    
#     tokens = nltk.word_tokenize(text)
    
#     # Add an extra occurrence for all-uppercase words with more than one letter
#     # temp = [word if (len(word) > 1 and word.isupper()) else None for word in tokens]
#     temp = []
#     for word in tokens:
#         if len(word) > 1 and word.isupper():
#             temp.append(word)
#     tokens.extend(x for x in temp if x)
    
#     # convert to lower case
#     tokens = [w.lower() for w in tokens]
#     # dr. = dr and st. = st and so on
#     tokens = [w.replace('.', '') for w in tokens]
    
#     # remove punctuation
#     # tokens = [word for word in tokens if word.isalpha()]
    
#     # Remove stop words
#     stop_words = set(stopwords.words("english"))
#     tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    
#     return tokens

# # print(train_data[4])
# # tokenize_text(train_data[4])


In [10]:
# tokens = []
# print(len(train_data))

# for i in range(len(train_data)):
#     tokens.append(tokenize_text(train_data[i]))
#     if i % 1000 == 0:
#         print(i)

# # save to pkl file
# with open('yelp_train_tokens_no_stop.pkl', 'wb') as file:
#     pickle.dump(tokens, file)

In [11]:
# import pickle
# from collections import Counter

# # Load the tokenized data
# with open('yelp_train_tokens_no_stop.pkl', 'rb') as file:
#     tokens = pickle.load(file)

# # Define the frequency cutoff threshold
# frequency_cutoff = 10

# # Count word frequencies
# word_counts = Counter(word for tokens_list in tokens for word in tokens_list)

# # Filter out words with counts less than the threshold
# filtered_tokens = [[word for word in tokens_list if word_counts[word] >= frequency_cutoff] for tokens_list in tokens]

# # Save the filtered tokens to a new file
# with open('yelp_train_tokens_filtered.pkl', 'wb') as file:
#     pickle.dump(filtered_tokens, file)

In [12]:
# tokens_test = []
# print(len(test_data))

# for i in range(len(test_data)):
#     tokens_test.append(tokenize_text(test_data[i]))
#     if i % 1000 == 0:
#         print(i)

# with open('yelp_test_tokens_no_stop.pkl', 'wb') as file:
#     pickle.dump(tokens_test, file)

In [13]:
# import pickle
# from collections import Counter

# # Load the tokenized data
# with open('yelp_test_tokens_no_stop.pkl', 'rb') as file:
#     tokens = pickle.load(file)

# # Define the frequency cutoff threshold
# frequency_cutoff = 10

# # Count word frequencies
# word_counts = Counter(word for tokens_list in tokens for word in tokens_list)

# # Filter out words with counts less than the threshold
# filtered_tokens = [[word for word in tokens_list if word_counts[word] >= frequency_cutoff] for tokens_list in tokens]

# # Save the filtered tokens to a new file
# with open('yelp_test_tokens_filtered.pkl', 'wb') as file:
#     pickle.dump(filtered_tokens, file)

In [14]:
import numpy as np

train_tokens = []
with open('yelp_train_tokens_filtered.pkl', 'rb') as file:
    train_tokens = pickle.load(file)

test_tokens = []
with open('yelp_test_tokens_filtered.pkl', 'rb') as file:
    test_tokens = pickle.load(file)
    
print("Loaded tokens")



Loaded tokens


In [15]:
# in train data replace every 1000th word with UNK randomly

import random

for i in range(len(train_tokens)):
    for j in range(len(train_tokens[i])):
        if random.randint(1, 1000) == 1:
            train_tokens[i][j] = 'UNK'
        

In [16]:
# Build the BoW representation manually
# Create a vocabulary by collecting unique words from the training data
vocab = set()
for tokens in train_tokens:
    vocab.update(tokens)

# Create a dictionary to map words to indices in the vocabulary
vocab_dict = {}
for i, word in enumerate(vocab):
    vocab_dict[word] = i

print(len(vocab_dict))
# Initialize BoW matrices for training and testing data


45287


In [17]:
# reduced_train_tokens = train_tokens

# # Build the BoW representation manually
# # Create a vocabulary by collecting unique words from the training data
# vocab_reduced = set()
# for tokens in reduced_train_tokens:
#     vocab_reduced.update(tokens)

# # Create a dictionary to map words to indices in the vocabulary
# vocab_dict_reduced = {}
# for i, word in enumerate(vocab_reduced):
#     vocab_dict_reduced[word] = i

# print(len(vocab_dict_reduced))

# train_bow = np.zeros((len(reduced_train_tokens), len(vocab_reduced)))

# print("bow train")
# # Convert text to BoW vectors
# for i, tokens in enumerate(reduced_train_tokens):
#     if i % 5000 == 0:
#         print(i)
#     for token in tokens:
#         train_bow[i][vocab_dict_reduced[token]] += 1



# reduced_test_tokens = test_tokens[:5000]
# test_bow = np.zeros((len(reduced_test_tokens), len(vocab_reduced)))

# print("bow test")

# for i, tokens in enumerate(reduced_test_tokens):
#     if i % 1000 == 0:
#         print(i)
#     for token in tokens:
#         if token in vocab_dict_reduced:
#             test_bow[i][vocab_dict_reduced[token]] += 1
#         else:    # if there is an unknown word, add it to the UNK column 
#             test_bow[i][vocab_dict_reduced['UNK']] += 1

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim

num_epochs = 4
learning_rate = 0.01
batch_size = 10000

class BoWClassifier(nn.Module):
    def __init__(self, input_size, output_size):
        super(BoWClassifier, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        # Pass the input through the linear layer
        out = self.linear(x)
        return out
    
# Define the model
input_size = len(vocab)  # Input size is the size of the vocabulary
output_size = 5  # Output size is 5 dimensions

model = BoWClassifier(input_size, output_size).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [19]:
import numpy as np
from collections import Counter
from math import log

def calculate_tfidf(tokens, vocab_dict):

    n_docs = len(tokens)
    vocab_size = len(vocab_dict)
    
    # Calculate document frequencies (DF) for each word
    doc_freq = Counter()
    for doc in tokens:
        doc_freq.update(set(doc))
    
    # Precompute IDF values
    idf_dict = {}
    for word, count in doc_freq.items():
        idf_dict[word] = log(n_docs / (count + 1))  # Add 1 to avoid division by zero
    
    tfidf_matrix = np.zeros((n_docs, vocab_size))
    
    for i, doc in enumerate(tokens):
        # if i % 1000 == 0:
        #     print(i)
        
        total_words_in_doc = len(doc)
        term_freq = Counter(doc)  # Calculate term frequency (TF) for the document
        
        for word, tf in term_freq.items():
            if word in vocab_dict:
                tfidf_matrix[i][vocab_dict[word]] = (tf / total_words_in_doc) * idf_dict[word]
            else:
                # Use 'UNK' if the word is not in the vocabulary
                tfidf_matrix[i][vocab_dict['UNK']] = (tf / total_words_in_doc) * idf_dict[word]
    
    return tfidf_matrix

In [20]:
# Training loop
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i in range(0, len(train_tokens), batch_size):
        
        inputs = train_tokens[i:i + batch_size]
        # print(len(inputs))
        # print(len(vocab))
        
        # make bow vector for inputs
        # bow = torch.zeros((batch_size, len(vocab)), dtype=torch.float32)
        # print(bow.shape)
        # for j in range(batch_size):
        #     for token in inputs[j]:

        #         bow[j][vocab_dict[token]] += 1

        bow = calculate_tfidf(inputs, vocab_dict)
        
        # convert bow to tensor
        inputs = torch.tensor(bow, dtype=torch.float32).to(device)
        
        labels = train_labels[i:i + batch_size]  # Make sure to have train_labels defined
        # Convert labels to LongTensors
        labels = torch.tensor(labels, dtype=torch.long).to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Calculate the loss
        loss = criterion(outputs, labels)

        # Backpropagation
        loss.backward()

        # Update weights
        optimizer.step()

        # calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


        # Print the loss for this batch if needed
        print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{i//batch_size+1}], Loss: {loss.item()}')

    print(f'Accuracy of the network on the {total} train inputs: {100 * correct / total} %')

print('Training finished')

Epoch [1/4], Batch [1], Loss: 1.6093493700027466
Epoch [1/4], Batch [2], Loss: 1.6014455556869507
Epoch [1/4], Batch [3], Loss: 1.5965968370437622
Epoch [1/4], Batch [4], Loss: 1.5862528085708618
Epoch [1/4], Batch [5], Loss: 1.5796576738357544
Epoch [1/4], Batch [6], Loss: 1.5670703649520874
Epoch [1/4], Batch [7], Loss: 1.5609441995620728
Epoch [1/4], Batch [8], Loss: 1.5536936521530151
Epoch [1/4], Batch [9], Loss: 1.552130937576294
Epoch [1/4], Batch [10], Loss: 1.5381931066513062
Epoch [1/4], Batch [11], Loss: 1.5426872968673706
Epoch [1/4], Batch [12], Loss: 1.5283254384994507
Epoch [1/4], Batch [13], Loss: 1.5296149253845215
Epoch [1/4], Batch [14], Loss: 1.5211189985275269
Epoch [1/4], Batch [15], Loss: 1.5048354864120483
Epoch [1/4], Batch [16], Loss: 1.5399950742721558
Epoch [1/4], Batch [17], Loss: 1.5265430212020874
Epoch [1/4], Batch [18], Loss: 1.5238316059112549
Epoch [1/4], Batch [19], Loss: 1.5263962745666504
Epoch [1/4], Batch [20], Loss: 1.5102314949035645
Epoch [1/4

In [21]:
# save the above model in its entirety in pkl file
torch.save(model, 'bow_model.pkl')

# load the model
model = torch.load('bow_model.pkl')

In [22]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, classification_report

# Test the model
with torch.no_grad():
    total = 0
    correct = 0
    predictions = []
    true_labels = []

    for i in range(0, len(test_tokens), batch_size):
        inputs = test_tokens[i:i + batch_size]
        # bow = torch.zeros((batch_size, len(vocab)), dtype=torch.float32)

        # for j in range(batch_size):
        #     for token in inputs[j]:
        #         if token in vocab_dict:
        #             bow[j][vocab_dict[token]] += 1
        #         else:
        #             bow[j][vocab_dict['UNK']] += 1

        bow = calculate_tfidf(inputs, vocab_dict)

        inputs = torch.tensor(bow, dtype=torch.float32).to(device)
        labels = test_labels[i:i + batch_size]
        labels = torch.tensor(labels, dtype=torch.long).to(device)

        outputs = model(inputs)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Collect predictions and true labels
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

    # Calculate accuracy
    accuracy = 100 * correct / total
    print(f'Accuracy of the network on the {total} test inputs: {accuracy} %')

    # Calculate confusion matrix, precision, recall, and F1 scores
    confusion = confusion_matrix(true_labels, predictions)
    classification_rep = classification_report(true_labels, predictions, target_names=[str(i) for i in range(5)])
    f1_micro = f1_score(true_labels, predictions, average='micro')
    f1_macro = f1_score(true_labels, predictions, average='macro')
    f1_weighted = f1_score(true_labels, predictions, average='weighted')
    precision = precision_score(true_labels, predictions, average='weighted')
    recall = recall_score(true_labels, predictions, average='weighted')

    print("Confusion Matrix:")
    print(confusion)
    print("Classification Report:")
    print(classification_rep)
    # print("F1 Micro:", f1_micro)
    # print("F1 Macro:", f1_macro)
    # print("F1 Weighted:", f1_weighted)
    # print("Precision:", precision)
    # print("Recall:", recall)


Accuracy of the network on the 50000 test inputs: 55.654 %
Confusion Matrix:
[[7396 1948  237   73  346]
 [2648 4757 1827  294  474]
 [ 838 2153 4189 1768 1052]
 [ 321  473 1719 3886 3601]
 [ 324  132  304 1641 7599]]
Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.74      0.69     10000
           1       0.50      0.48      0.49     10000
           2       0.51      0.42      0.46     10000
           3       0.51      0.39      0.44     10000
           4       0.58      0.76      0.66     10000

    accuracy                           0.56     50000
   macro avg       0.55      0.56      0.55     50000
weighted avg       0.55      0.56      0.55     50000

